In [2]:
import numpy
from numpy import *
from scipy import optimize
import csv
import pandas as pd
df1=pd.read_csv("reviews.csv")
df2=pd.read_csv("names.csv")

#fill zeros for missing values
df1=df1.fillna(0)

#convert user ratings (ratings) to matrix
ratings = df1.as_matrix()

did_rate = (ratings != 0) * 1

num_movies = 9066
num_users = 671

#create array of user ratings
ben_ratings = zeros((num_movies, 1))
ben_ratings.shape

#user input for movies
x11 = int(input('Enter Movie 1 ID : '))
x12 = int(input('Enter Movie 1 Score : '))

x21 = int(input('Enter Movie 2 ID : '))
x22 = int(input('Enter Movie 2 Score : '))

x31 = int(input('Enter Movie 3 ID : '))
x32 = int(input('Enter Movie 3 Score : '))

x41 = int(input('Enter Movie 4 ID : '))
x42 = int(input('Enter Movie 4 Score : '))

x51 = int(input('Enter Movie 5 ID : '))
x52 = int(input('Enter Movie 5 Score : '))

ben_ratings[x11] = x12
ben_ratings[x21] = x22
ben_ratings[x31] = x32
ben_ratings[x41] = x42
ben_ratings[x51] = x52

# Updating the ratings and did_rate
ratings = append(ben_ratings, ratings, axis = 1)
did_rate = append(((ben_ratings != 0) * 1), did_rate, axis = 1)

# Function that normalizes a dataset
def normalize_ratings(ratings, did_rate):
    num_movies = ratings.shape[0]
    ratings_mean = zeros(shape = (num_movies, 1))
    ratings_norm = zeros(shape = ratings.shape)
    for i in range(num_movies): 
        # Get all the indexes where there is a 1
        idx = where(did_rate[i] == 1)[0]
        #  Calculate mean rating of ith movie only from user's that gave a rating
        ratings_mean[i] = mean(ratings[i, idx])
        ratings_norm[i, idx] = ratings[i, idx] - ratings_mean[i]
    
    return ratings_norm, ratings_mean


# Normalizing our ratings
ratings, ratings_mean = normalize_ratings(ratings, did_rate)

# Update some key variables now
num_users = ratings.shape[1]
num_features = 10

# Initialize Parameters theta (user_prefs), X (movie_features)
movie_features = random.randn(num_movies, num_features)
user_prefs = random.randn(num_users, num_features)
initial_X_and_theta = r_[movie_features.T.flatten(), user_prefs.T.flatten()]

def unroll_params(X_and_theta, num_users, num_movies, num_features):
# Retrieve the X and theta matrixes from X_and_theta, based on their dimensions (num_features, num_movies, num_movies)
# --------------------------------------------------------------------------------------------------------------
# Get the first 30 (10 * 3) rows in the 48 X 1 column vector
    first_30 = X_and_theta[:num_movies * num_features]
# Reshape this column vector into a 10 X 3 matrix
    X = first_30.reshape((num_features, num_movies)).transpose()
# Get the rest of the 18 the numbers, after the first 30
    last_18 = X_and_theta[num_movies * num_features:]
# Reshape this column vector into a 6 X 3 matrix
    theta = last_18.reshape(num_features, num_users ).transpose()
    return X, theta

def calculate_gradient(X_and_theta, ratings, did_rate, num_users, num_movies, num_features, reg_param):
    X, theta = unroll_params(X_and_theta, num_users, num_movies, num_features)

# we multiply by did_rate because we only want to consider observations for which a rating was given
    difference = X.dot( theta.T ) * did_rate - ratings
    X_grad = difference.dot( theta ) + reg_param * X
    theta_grad = difference.T.dot( X ) + reg_param * theta
# wrap the gradients back into a column vector 
    return r_[X_grad.T.flatten(), theta_grad.T.flatten()]

def calculate_cost(X_and_theta, ratings, did_rate, num_users, num_movies, num_features, reg_param):
    X, theta = unroll_params(X_and_theta, num_users, num_movies, num_features)

# we multiply (element-wise) by did_rate because we only want to consider observations for which a rating was given
    cost = sum( (X.dot( theta.T ) * did_rate - ratings) ** 2 ) / 2
# '**' means an element-wise power
    regularization = (reg_param / 2) * (sum( theta**2 ) + sum(X**2))
    return cost + regularization

# regularization paramater
reg_param = 30

# perform gradient descent, find the minimum cost (sum of squared errors) and optimal values of X (movie_features) and Theta (user_prefs)
minimized_cost_and_optimal_params = optimize.fmin_cg(calculate_cost, fprime=calculate_gradient, x0=initial_X_and_theta,
args=(ratings, did_rate, num_users, num_movies, num_features, reg_param),maxiter=1000, disp=True, full_output=True)

cost, optimal_movie_features_and_user_prefs = minimized_cost_and_optimal_params[1], minimized_cost_and_optimal_params[0]
movie_features, user_prefs = unroll_params(optimal_movie_features_and_user_prefs, num_users, num_movies, num_features)

# Make some predictions (movie recommendations). Dot product
all_predictions = movie_features.dot( user_prefs.T )

# add back the ratings_mean column vector to my (our) predictions
predictions_for_ahsen = all_predictions[:, 0:1] + ratings_mean

recommendations=pd.DataFrame(index=range(0,num_movies),columns=['Score','Movie_Name'])
recommendations['Score'] = pd.DataFrame(predictions_for_ahsen)
recommendations['Movie_Name'] = df2

recommendations.index.name = 'Movie_ID'
recommendations = recommendations.sort_values(['Score'], ascending=[False])
recommendations.to_csv('recommendations.csv')

print('================================================================')
print('Your movie recommendations are ready! Check the recommendations.csv output file.')
print('================================================================')
print(recommendations.head(10))

#Ratings
#Braveheart (100) = 5
#Malcolm X (2602) = 5
#Avatar (7388) = 4
#Batman V.s Superman (8895) = 2
#Twilight (7070) = 1


Enter Movie 1 ID : 100
Enter Movie 1 Score : 5
Enter Movie 2 ID : 2602
Enter Movie 2 Score : 5
Enter Movie 3 ID : 7388
Enter Movie 3 Score : 4
Enter Movie 4 ID : 8895
Enter Movie 4 Score : 2
Enter Movie 5 ID : 7070
Enter Movie 5 Score : 1
Optimization terminated successfully.
         Current function value: 38781.960509
         Iterations: 116
         Function evaluations: 178
         Gradient evaluations: 178
Your movie recommendations are ready! Check the recommendations.csv output file.
          Score                                         Movie_Name
Movie_ID                                                          
3155        5.0                                  Get Carter (1971)
1252        5.0                                      Gabbeh (1996)
1233        5.0                     Designated Mourner, The (1997)
8997        5.0                           World of Tomorrow (2015)
8639        5.0                              Helter Skelter (1976)
3499        5.0                 